## Project Description

## Notebook Description

### Initialize packages and read in pickled data

In [1]:
% run __init__.py

In [3]:
cd ..

/home/jovyan


In [4]:
df_model = pd.read_pickle('data/df_model.p')

In [5]:
df_model.shape

(127052, 88)

### Set up df to `run_benchmark` function on standardized data and compare against un-standardized

In [6]:
df_model.drop('player_id', axis=1, inplace=True)

In [7]:
target = df_model['hit_flag']
predictors = df_model.drop('hit_flag', axis=1)

In [8]:
def run_benchmark(model, model_name, dataframe, target_col):
    target = dataframe[target_col]
    tmp_df = dataframe.drop(target_col, axis=1)
    X_train, X_test, y_train, y_test = train_test_split(tmp_df, target)
    model.fit(X_train, y_train)
    return {'train_score' : model.score(X_train, y_train), 
            'test_score' : model.score(X_test, y_test),
            'model_name' : model_name }

# credit to Joshua Cook

### Standardize Features

In [9]:
df_model_st_all = predictors.copy()
# df_model_st_num = predictors.copy()

**`StandardScaler()` on All Features**

In [10]:
standardized = (StandardScaler().fit_transform(df_model_st_all))
df_standardized = pd.DataFrame(standardized, columns=df_model_st_all.columns, index=df_model_st_all.index)

In [11]:
df_standardized.shape

(127052, 86)

In [12]:
target.shape

(127052,)

In [13]:
df_model_st = pd.concat([df_standardized, target], axis=1)

**`StandardScaler()` on Numerical Features**

Maybe...

### Run Models on Standardized Data

### K Neighbors Classifier

In [18]:
knn_output = run_benchmark(KNeighborsClassifier(),
                           'kneighbors',
                           df_model_st, 
                           'hit_flag')

In [19]:
knn_output

{'model_name': 'kneighbors',
 'test_score': 0.64502723294399145,
 'train_score': 0.76942774087250365}

### Logistic Regression

In [14]:
log_reg_output = run_benchmark(LogisticRegression(), 
                               'logistic regression',
                               df_model_st, 
                               'hit_flag')

In [15]:
log_reg_output

{'model_name': 'logistic regression',
 'test_score': 0.72049239681390298,
 'train_score': 0.7209961275698139}

### Decision Tree

In [16]:
dtree_output = run_benchmark(DecisionTreeClassifier(), 
                             'decision tree',
                             df_model_st, 
                             'hit_flag')

In [17]:
dtree_output

{'model_name': 'decision tree',
 'test_score': 0.75830368667946979,
 'train_score': 1.0}

### Show benchmark models side-by-side

In [20]:
output = [
    knn_output,
    log_reg_output,
    dtree_output
]

pd.DataFrame(output)

,model_name,test_score,train_score
0,kneighbors,0.645027,0.769428
1,logistic regression,0.720492,0.720996
2,decision tree,0.758304,1.000000


While logistic regression and decision tree perform slightly better with `StandardScaler` normalization, K neighbors performs significantly worse. `StandardScaler` is forcing every feature to a mean of 0 with a standard deviation of 1. Based on EDA, I believe there are some very unimportant features in the data that have become more noisy as a result of normalization. However, my hypothesis is this will make it easier to parse them out in feature selection later on.